In [1]:
#   Name: Teresa Ferrill
#   Date: 04/19/2020
# Course: DSC540-T302 Data Preparation
# Term Project: Milestone Two - Gun Violence

Cleaning/Formatting Flat File Source

Perform at least 5 data transformation and/or cleansing steps to your flat file data.  

> 1. Replace Headers     
> 2. Identify outliers and bad data   
> 3. Find duplicates    
> 4. Format data into a more readable format    
> 5. Fix casing or inconsistent values    

In [2]:
# Start with importing file and reviewing its contents

# import libraries used within Notebook
import pandas as pd
import numpy as np

# import csv file into DataFrame and review its contents
guns = pd.read_csv('guns.csv')

# preserve number of rows of original DataFrame
originalRowCount = guns.shape[0]

# obtain number of rows and columns within the file
print(guns.shape)

# print first 5 rows of DataFrame
guns.head()

(100798, 11)


,Unnamed: 0,year,month,intent,police,sex,age,race,hispanic,place,education
0,1,2012,1,Suicide,0,M,34.0,Asian/Pacific Islander,100,Home,4.0
1,2,2012,1,Suicide,0,F,21.0,White,100,Street,3.0
2,3,2012,1,Suicide,0,M,60.0,White,100,Other specified,4.0
3,4,2012,2,Suicide,0,M,64.0,White,100,Home,4.0
4,5,2012,2,Suicide,0,M,31.0,White,100,Other specified,2.0


Column information:
> __Unnamed:0__: this is an identifier column, which contains the row number   
> __year__: the year in which the fatality occurred      
> __month__: the month in which the fatality occurred    
> __intent__: the intent of the perpetrator of the crime, including suicide, accidental, NA, homicide, or undetermined      
> __police__: whether a police officer was involved with the shooting, either 0 (false) or 1 (true)     
> __sex__: the gender of the victim, either M or F       
> __age__: the age of the victim     
> __race__: the race of the victim, including Asian/Pacific Islander, Native American/Native Alaskan, Black, Hispanic, or White     
> __hispanic__: a code indicating the Hispanic origin of the victim   
>> Unable to determine the description of each code       
>>
> __place__: where the shooting occurred, multiple categories      
> __education__: educational status of the victim, one of the following:      
>> 1: Less than High School       
>> 2: Graduated from High School or equivalent       
>> 3: Some College        
>> 4: At least graduated from College       
>> 5: Not available

In [3]:
# 1. Replace Headers

# added header to row index column
guns.index.name = 'Index'

# renamed Unnamed column to Identifier
guns = guns.rename(columns = {'Unnamed: 0':'Identifier'})

# print first 5 rows of DataFrame
guns.head()

,Identifier,year,month,intent,police,sex,age,race,hispanic,place,education
Index,,,,,,,,,,,
0,1,2012,1,Suicide,0,M,34.0,Asian/Pacific Islander,100,Home,4.0
1,2,2012,1,Suicide,0,F,21.0,White,100,Street,3.0
2,3,2012,1,Suicide,0,M,60.0,White,100,Other specified,4.0
3,4,2012,2,Suicide,0,M,64.0,White,100,Home,4.0
4,5,2012,2,Suicide,0,M,31.0,White,100,Other specified,2.0


In [4]:
# for readability and concistency - capitalizing column names
guns.columns = map(str.capitalize, guns.columns)

# list column headers
guns.columns

Index(['Identifier', 'Year', 'Month', 'Intent', 'Police', 'Sex', 'Age', 'Race',
       'Hispanic', 'Place', 'Education'],
      dtype='object')

In [5]:
# 2. Identify outliers and bad data - focus is on bad data

# using dtypes, determine data types of each column 
# looking for column types that could point to rows without data, etc.
guns.dtypes

Identifier      int64
Year            int64
Month           int64
Intent         object
Police          int64
Sex            object
Age           float64
Race           object
Hispanic        int64
Place          object
Education     float64
dtype: object

The float values in the education column could indicate there are NaN values, since every category is a whole number [1,2,3,4,5]. 

In [6]:
# determine where NaN values exist

# notnull().sum() method counts the rows within the dataset that do not contain any null values
guns.notnull().sum()

Identifier    100798
Year          100798
Month         100798
Intent        100797
Police        100798
Sex           100798
Age           100780
Race          100798
Hispanic      100798
Place          99414
Education     100745
dtype: int64

Results indiate there are null values in the Intent, Age, Place, and Education columns.

In [7]:
# checking percentage of valid data, by column
guns.notnull().sum() * 100.0/guns.shape[0]

Identifier    100.000000
Year          100.000000
Month         100.000000
Intent         99.999008
Police        100.000000
Sex           100.000000
Age            99.982143
Race          100.000000
Hispanic      100.000000
Place          98.626957
Education      99.947420
dtype: float64

Most of the columns have at least 98.6% of the values - which means the data is close to complete.  Deleting all rows with NaN values should not impact any insights and conclusions.

In [8]:
# drop rows with NaN values 
guns = guns.dropna(subset = ['Intent', 'Age', 'Place', 'Education'])

# preserve number of rows of updated DataFrame
updatedRowCount = guns.shape[0]

# print number of rows and columns after dropping NaN values
print(guns.shape)

# checking percentage of valid data again
guns.notnull().sum() * 100.0/guns.shape[0]

(99343, 11)


Identifier    100.0
Year          100.0
Month         100.0
Intent        100.0
Police        100.0
Sex           100.0
Age           100.0
Race          100.0
Hispanic      100.0
Place         100.0
Education     100.0
dtype: float64

In [9]:
# Determine number of deleted rows, compute difference between original and updated DataFrame
print('Number of deleted rows: {}.'.format(originalRowCount - updatedRowCount))

Number of deleted rows: 1455.


In [10]:
# 3. find duplicates

# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = guns[guns.duplicated()]

# print results
print("Duplicate Rows except based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except based on all columns are :
Empty DataFrame
Columns: [Identifier, Year, Month, Intent, Police, Sex, Age, Race, Hispanic, Place, Education]
Index: []


The Empty DataFrame indicates there are no duplicates in this DataFrame.

In [11]:
# 4. Format data into a more readable format
# modify Education column to reflect description instead of a number
# modify Police column to reflect 'True' or 'False' instead of 0 or 1

# modify the Education and Police column to a string type (reflects as object)
guns['Education'] = guns['Education'].astype(str)
guns['Police'] = guns['Police'].astype(str)
guns.dtypes

Identifier      int64
Year            int64
Month           int64
Intent         object
Police         object
Sex            object
Age           float64
Race           object
Hispanic        int64
Place          object
Education      object
dtype: object

In [12]:
# update Education and Police columns based on entry

# set Education column value based on current value
guns['Education']=np.where((guns.Education == '1.0'), 'Less than High School', guns.Education)
guns['Education']=np.where((guns.Education == '2.0'), 'Graduated from High School or equivalent', guns.Education)
guns['Education']=np.where((guns.Education == '3.0'), 'Some college', guns.Education)
guns['Education']=np.where((guns.Education == '4.0'), 'At least graduated from college', guns.Education)
guns['Education']=np.where((guns.Education == '5.0'), 'Not available', guns.Education)

# set Police column based on current value
guns['Police']=np.where((guns.Police == '0'), 'False', guns.Police)
guns['Police']=np.where((guns.Police == '1'), 'True', guns.Police)

# print first 5 rows of DataFrame
guns.head()

,Identifier,Year,Month,Intent,Police,Sex,Age,Race,Hispanic,Place,Education
Index,,,,,,,,,,,
0,1,2012,1,Suicide,False,M,34.0,Asian/Pacific Islander,100,Home,At least graduated from college
1,2,2012,1,Suicide,False,F,21.0,White,100,Street,Some college
2,3,2012,1,Suicide,False,M,60.0,White,100,Other specified,At least graduated from college
3,4,2012,2,Suicide,False,M,64.0,White,100,Home,At least graduated from college
4,5,2012,2,Suicide,False,M,31.0,White,100,Other specified,Graduated from High School or equivalent


In [13]:
# 5. Fix casing or inconsistent values
# Check non-numeric columns for consistent values - Intent, Sex, Race, Place, Education, Police
# Checking Education and Police in case there were unexpected values prior to update

# list unique values in the Intent column
guns.Intent.unique()

array(['Suicide', 'Undetermined', 'Accidental', 'Homicide'], dtype=object)

In [14]:
#List unique values in the Intent column
guns.Intent.unique()

array(['Suicide', 'Undetermined', 'Accidental', 'Homicide'], dtype=object)

In [15]:
# List unique values in the Sex column
guns.Sex.unique()

array(['M', 'F'], dtype=object)

In [16]:
# List unique values in the Race column
guns.Race.unique()

array(['Asian/Pacific Islander', 'White',
       'Native American/Native Alaskan', 'Black', 'Hispanic'],
      dtype=object)

In [17]:
# List unique values in the Place column
guns.Place.unique()

array(['Home', 'Street', 'Other specified', 'Other unspecified',
       'Trade/service area', 'Farm', 'Industrial/construction',
       'School/instiution', 'Sports', 'Residential institution'],
      dtype=object)

In [18]:
# List unique values in the Education column
guns.Education

array(['At least graduated from college', 'Some college',
       'Graduated from High School or equivalent',
       'Less than High School', 'Not available'], dtype=object)

In [19]:
# List unique values in the Police column
guns.Police.unique()

array(['False', 'True'], dtype=object)

Based on results, all values in the Intent, Sex, Race, Place, Education, and Police columns are consistent

Reference

>> Kaggle. (n.d.). Gun Violence. Retrieved from Kaggle: https://www.kaggle.com/hakabuk/gun-deaths-in-the-us